In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

class CatDogClassifier:
    def __init__(self, img_height=224, img_width=224):
        self.img_height = img_height
        self.img_width = img_width
        self.model = None
        
    def create_model(self, num_classes=2):
        """Create a model using transfer learning with MobileNetV2"""
        # Load pre-trained MobileNetV2
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=(self.img_height, self.img_width, 3),
            include_top=False,
            weights='imagenet'
        )
        
        # Freeze the base model
        base_model.trainable = False
        
        # Add custom layers on top
        model = tf.keras.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dropout(0.2),
            layers.Dense(128, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(num_classes, activation='softmax')
        ])
        
        # Compile the model
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        self.model = model
        return model
    
    def prepare_data(self, data_dir, batch_size=32, validation_split=0.2):
        """Prepare data using ImageDataGenerator"""
        # Data augmentation for training
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            validation_split=validation_split
        )
        
        # Only rescaling for validation
        val_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=validation_split
        )
        
        # Train generator
        train_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=batch_size,
            class_mode='binary',
            subset='training'
        )
        
        # Validation generator
        val_generator = val_datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=batch_size,
            class_mode='binary',
            subset='validation'
        )
        
        return train_generator, val_generator
    
    def train(self, train_generator, val_generator, epochs=10):
        """Train the model"""
        if self.model is None:
            self.create_model()
        
        # Callbacks
        callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=2)
        ]
        
        # Train the model
        history = self.model.fit(
            train_generator,
            epochs=epochs,
            validation_data=val_generator,
            callbacks=callbacks
        )
        
        return history
    
    def predict(self, image_path):
        """Predict if image is cat or dog"""
        if self.model is None:
            raise ValueError("Model not trained yet!")
        
        # Load and preprocess image
        img = tf.keras.preprocessing.image.load_img(
            image_path, target_size=(self.img_height, self.img_width)
        )
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create batch dimension
        img_array /= 255.0  # Normalize
        
        # Make prediction
        predictions = self.model.predict(img_array)
        class_idx = np.argmax(predictions[0])
        confidence = np.max(predictions[0])
        
        classes = {0: 'cat', 1: 'dog'}
        
        return classes[class_idx], confidence
    
    def save_model(self, filepath):
        """Save the trained model"""
        if self.model is not None:
            self.model.save(filepath)
    
    def load_model(self, filepath):
        """Load a pre-trained model"""
        self.model = tf.keras.models.load_model(filepath)

# Usage example
def main():
    # Initialize classifier
    classifier = CatDogClassifier()
    
    # Create model
    model = classifier.create_model()
    print(model.summary())
    
    # Prepare data (assuming your data is in this structure)
    # data_dir/
    #   train/
    #     cats/
    #     dogs/
    #   validation/
    #     cats/
    #     dogs/
    
    data_dir = "/Python_ML/deep/trainset.hdf5"
    train_gen, val_gen = classifier.prepare_data(data_dir)
    
    # Train the model
    history = classifier.train(train_gen, val_gen, epochs=15)
    
    # Save the model
    classifier.save_model("cat_dog_classifier.h5")
    
   
if __name__ == "__main__":
    main()

2025-11-04 11:25:41.457677: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 11:25:54.195095: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/johary/anaconda3/envs/tensorflow-env/lib/python3.9/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
2025-11-04 11:26:01.453016: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 11:26:07.139534: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit:

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5: None -- [Errno -3] Temporary failure in name resolution

In [ ]:
 # Make prediction on a new image
    prediction, confidence = classifier.predict("path/to/your/image.jpg")
    print(f"Prediction: {prediction}, Confidence: {confidence:.2f}")
